In [5]:
from arcgis.gis import GIS, Item
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')
sys.path.append('../')

import reach_tools
import hydrology
import config

gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

url_reach_line = config.url_reach_line
url_reach_centroid = config.url_reach_centroid
url_reach_points = config.url_reach_points

lyr_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

reach_id = 2127

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
reach_id_fs = lyr_centroid.query(out_fields='reach_id', result_record_count=20, return_geometry=False)
reach_id_lst = [int(feat.attributes['reach_id']) for feat in reach_id_fs.features]
print(reach_id_lst[151:])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [108]:
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

for reach_id in reach_id_lst[:43]:
    
    reach = reach_tools.Reach.get_from_aw(reach_id)
    reach.snap_putin_and_takeout_and_trace(gis_esri)
    reach.publish_updates(lyr_line, lyr_centroid, lyr_points)
        
    print(f'success: {reach.reach_id} {reach.river_name} {reach.reach_name} http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid={reach.reach_id}')

Enter password:  ··········


RuntimeError: Invalid token

In [92]:
reach.putin.subtype

'putin'

In [83]:
reach = reach_tools.Reach.get_from_aw(3066)

In [84]:
print(reach.abstract)

A classic creek run with a series of outstanding drops. Has been the site of the Canyon Creek Extreme Race.


In [87]:
reach.as_feature.attributes

{'abstract': 'A classic creek run with a series of outstanding drops. Has been the site of the Canyon Creek Extreme Race.',
 'agency': None,
 'centroid': {'x': -122.3445014953615,
  'y': 45.95024871826175,
  'spatialReference': {'wkid': 4326}},
 'description': 'Events: Annual Canyon Creek Extreme Race Logistics: The take-out is at the base of the Merwin Reservoir bridge at mile 28.8 on Hwy. 503. Ample parking is available on the southeast side of the bridge (PacifiCorp has plans to make minor improvements to this site over the next couple years which will likely include a set of stairs and some improved drainage for the parking lot). There are a couple ways to get there depending on the direction you\'re coming from, but it\'s 25 miles east of I-5. To reach the put-in head south from the bridge to Hwy. 503 mile 24.8 and turn east on to NE Healy Rd. (becomes FR 54) and drive 3.9 miles to the bridge across Canyon Creek. Put-in on upstream river left where a jeep trail leads down to the r

In [82]:
val = reach._reach_json['gauges'][0]
val

{'dhid': 40247,
 'id': 40247,
 'did': 40247,
 'gauge_name': 'CANYON CREEK NEAR AMBOY, WA',
 'updated': '3270.59474',
 'epoch': '3270.59474',
 'reading': '255',
 'gauge_metric': 2,
 'gauge_min': '500.0000',
 'gauge_max': '700.0000',
 'obs_id': 153180203151115,
 'gauge_id': 35665,
 'gauge_reading': '255',
 'last_gauge_reading': '255',
 'last_gauge_updated': '3584',
 'gauge_comment': None,
 'gauge_data': None,
 'range_comment': '',
 'gauge_perfect': False,
 'gauge_estimated': False,
 'gauge_important': False,
 'range_min': 'R0',
 'range_max': 'R5',
 'adjusted_reach_class': None,
 'targetid': 40247,
 'sourceid': 35665,
 'metricid': 2,
 'min': '500.0000',
 'max': '700.0000',
 'source': 'usgs',
 'source_id': '14219000',
 'glat': '45.939834',
 'glon': '-122.317040',
 'time_adjustment': 0,
 'class': 'low',
 'rc': -1.225,
 'excluded': False,
 'rmin': '1100.0000',
 'rmax': '1500.0000',
 'metric_unit': 'cfs',
 'url': '/content/Gauge2/detail/id/35665'}

In [59]:
from datetime import datetime

In [72]:
datetime.fromtimestamp(float(val)*100000)

datetime.datetime(1978, 4, 20, 20, 0, 40, 900000)